In [2]:
from random import randint
from hashlib import sha3_256, sha3_512

key_hash = sha3_512

def hash_j(j, bts):
    # Defines an independent hash function for index j
    return sha3_256(bts + str(j).encode()).digest()

def hbss_keygen(m, key_length):
    # Keygen function for classic HBSS scheme
    preimages = [[], []]
    commitements = [[], []]
    for j in range(m):
        preimages[0].append(randint(2**(key_length - 1), 2**key_length - 1))
        preimages[1].append(randint(2**(key_length - 1), 2**key_length - 1))
        commitements[0].append(key_hash(str(preimages[0][j]).encode()).digest())
        commitements[1].append(key_hash(str(preimages[1][j]).encode()).digest())

    return preimages, commitements

def hbss_sign(message, preimages, k, ij_used = None):
    # Signing function for classic HBSS scheme
    h = int(key_hash(message.encode()).hexdigest(), 16)
    D = [(h & 2**i) >> (i)  for i in range(h.bit_length())]
    m = len(preimages[0])

    j = 0
    while k > len(D):
        h = int(key_hash(str(j).encode() + message.encode()).hexdigest(), 16)
        D += [(h & 2**i) >> (i) for i in range(h.bit_length())]
        j +=1

    signature = []
    for j in range(k):
        i_j = int.from_bytes(hash_j(j, message.encode()), byteorder='big') % m
        if ij_used is not None:
            ij_used.append(i_j)
        signature.append(preimages[D[j]][i_j])

    return signature

def hbss_verify(message, signature, commitements, k):
    # Verification function for classic HBSS scheme
    h = int(key_hash(message.encode()).hexdigest(), 16)
    D = [(h & 2**i) >> (i) for i in range(h.bit_length())]
    m = len(commitements[0])

    j = 0
    while k > len(D):
        h = int(key_hash(str(j).encode() + message.encode()).hexdigest(), 16)
        D += [(h & 2**i) >> (i) for i in range(h.bit_length())]
        j +=1

    for j in range(k):
        i_j = int.from_bytes(hash_j(j, message.encode()), byteorder='big') % m
        if sha3_512(str(signature[j]).encode()).digest() != commitements[D[j]][i_j]:
            return False

    return True

def test_hbss():
    m = 1000
    key_length = 256
    k = 5
    message = "Hello, world!"

    preimages, commitements = hbss_keygen(m, key_length)
    print(preimages[0], preimages[1], sep='\n')
    print(commitements[0], commitements[1], sep='\n')
    signature = hbss_sign(message, preimages, k)
    print(signature)
    assert hbss_verify(message, signature, commitements, k)

test_hbss()

[...]
[b'\x9e\x13\xd1dBI\x0e&\xf6\x00\xb1\xe2m\x1a=D]\x9c\r\xd0\xcc\xc0\x04[<a\xc7A[\x1e|:\xe9"\xcb\x18:\\\xdf\xd4\xe0\xa6\x89\xe8\xf2&C\n\x8d\x84w\x85\x1f\xf8\xc0\xbd\x01\x95\x0e\x99\x13 \xc6\xb2', b'\xd9Ym\xc1\xf7Y:\xd4\xe4\x05<\x85?\x06\xc7\xb9\xaa\x987y\xc1\x18\xc5S\xdc)\x02\xcf\xa2\xf9\xa7HU\x83\xcf4a\x9c\xe1\xbb6r\xcb\x07v\x82r}\xff\xfc\x9b\x05$c\xbec\x9c\xcc\xe4\xba\xf7\x1bD\x03', b'\xdd\xb6\xbd\x0c\xdc\x96\x0e\x0b\x06\x9d\xdc\xd3VA\x7f\x8cW\xf9s\xbe\t\xad\xbb\xac\xae\x1d%Vn\x012G\xb9\xd6\x93\x89\xb8\xde\x024i\xc0\xfe\x97\x8c\x14\x91\x02\x8c\x16-\xa8[\x06\x05%\t\xcf&\x83S\xd9\xb9\xc5', b'"F\xf1\x13<Gm\xaeB\xe2\xc0\x17C*2\x82\xa03\xf4\t(\xc4\xef\x84|\xe7\xec\x0f\x8b\x15,\xfb9I\xe1\xdc\xd9\x0b\xde\xac\xd5\xea/H\x9ey)\xc7u\xb4\xf2\x17B\x14\xc4r\x01Lo\x80\xe4\xf7\xda\x88', b'b\xc5\xdf\xa5O^\xd9@\xcawf\xb9\x91\x06S|\xff\xd21#r2;\xce\xd1\xeb\xf2\x05J\xc6\x93\x01\xb5JxP\\{\xb7\xbd\x07\xc3\x1fW\xd4SO\x02]\xed\xde7C\xee\x94Y\xb9\x08i+\xe6\xa5\x82\x16', b'wk3Y\x1d\xb1z\xb3\x8e\x07A:\x82bN

In [3]:
from random import randint
from hashlib import sha3_256, sha3_512

key_hash = sha3_512

def hash_l_fold(l, bts):
    # hashes l times the input bytes
    for _ in range(l):
        bts = key_hash(bts).digest()
    return bts

def hash_j(j, bts):
    # Defines an independent hash function for index j
    return sha3_256(bts + str(j).encode()).digest()

def f(l, message, k, beta):
    # functionality for finding depths given a probability vector beta
    ds = []
    for j in range(k):
        S = sha3_256(b"depth" + str(j).encode() + message.encode())
        len = S.digest_size * 8
        S = int.from_bytes(S.digest(), byteorder='big')

        for d in range(1, l + 1):
            if S < sum(beta[:d]) * 2**(len) or d == l:
                ds.append(d)
                break
    return ds

def hbss_plus_keygen(m, l, key_length):
    # Keygen function for HBSS+ scheme
    preimages = [[], []]
    commitements = [[], []]
    for j in range(m):
        preimages[0].append(randint(2**(key_length - 1), 2**key_length - 1).to_bytes(key_length // 8, byteorder='big'))
        preimages[1].append(randint(2**(key_length - 1), 2**key_length - 1).to_bytes(key_length // 8, byteorder='big'))

        commitements[0].append(hash_l_fold(l, preimages[0][j]))
        commitements[1].append(hash_l_fold(l, preimages[1][j]))

    return preimages, commitements

def hbss_plus_sign(message, preimages, k, l, beta, ij_used = None):
    # Signing function for HBSS+ scheme
    h = int(key_hash(message.encode()).hexdigest(), 16)
    D = [(h & 2**i) >> (i)  for i in range(h.bit_length())]
    m = len(preimages[0])
    
    j = 0
    while k > len(D):
        h = int(key_hash(str(j).encode() + message.encode()).hexdigest(), 16)
        D += [(h & 2**i) >> (i) for i in range(h.bit_length())]
        j +=1
    
    signature = []
    ds = f(l, message, k, beta)
    for j in range(k):
        i_j = int.from_bytes(hash_j(j, message.encode()), byteorder='big') % m
        if ij_used is not None:
            ij_used.append((i_j, ds[j]))
        signature.append(hash_l_fold(l - ds[j], preimages[D[j]][i_j]))

    return signature

def hbss_plus_verify(message, signature, commitements, k, l, beta):
    # Verification function for HBSS+ scheme
    h = int(key_hash(message.encode()).hexdigest(), 16)
    D = [(h & 2**i) >> (i) for i in range(h.bit_length())]
    m = len(commitements[0])

    j = 0
    while k > len(D):
        h = int(key_hash(str(j).encode() + message.encode()).hexdigest(), 16)
        D += [(h & 2**i) >> (i) for i in range(h.bit_length())]
        j +=1

    ds = f(l, message, k, beta)
    for j in range(k):
        i_j = int.from_bytes(hash_j(j, message.encode()), byteorder='big') % m
        if hash_l_fold(ds[j], signature[j]) != commitements[D[j]][i_j]:
            return False

    return True
        
def test_hbss_plus():
    m = 10
    key_length = 256
    k = 10
    l = 3
    beta = [0.5, 0.25, 0.25]
    message = "Hello, world!"

    preimages, commitements = hbss_plus_keygen(m, l, key_length)
    print(preimages[0], preimages[1], sep='\n')
    print(commitements[0], commitements[1], sep='\n')
    signature = hbss_plus_sign(message, preimages, k, l, beta)
    print(signature)
    assert hbss_plus_verify(message, signature, commitements, k, l, beta)

test_hbss_plus()

[b'\xb6\x94\\\xa6P\xafGb\x01\xcf\x03J\xd2\xb2\x92%\xe2\x1c\xc9\xb6\xbc\xbe\xb2\xf2\x99#\xae\r\n\xbb\x9cv', b'\xef\xf3\x12\xe8\xcf\x86\xb7\x1b-\x9f\x11\xca\x8c\x0e\x03!Tf{\xa8\t0F\xfa0\x90^g6\x03\xf9?', b'\xb5\xc0\xfb\x93\xd3\tW\xa2\xde\xa0\xd7\x8c+F\x19.K\xc7\xe8\x16\xceE:n\x0b\xee\xf8AKq\x86\x87', b'\xff\xaa\xfbr\xb2\xf6+\x1c\xe2\xcf\x8b\xe7}\xb2\xc4\xdb5f\xe8M\\r\x10ap\x07\xce\xf6J\xed\xf4\x03', b'\xf8\xb36\x83\xeaP9],\xe9\xee\xa14\xf7\x91\xa5\x10/\x89!\x17\x18\x18b3*\xf4\x8b\xb0M\xad&', b'\x82\xa0\xbc\xe9\x92\x8c\xe0\xf8\xbc}X\x08y\x03\xfc,zk\xef\xf8\xc5e+w\xc7MY[i\x1a\xeb\xb5', b'\xd5\xe7\x12\x7f\x80y\xe9\xba\xcc\x03t>5\xb0\x1a\xb2\x89\xe5\xb8\xbacu\xad\x12\x14]d\x87=\x89#f', b'\xff\x19d\xe6N.\xb4&7\xd4$2\xe9\xea\xe7\xa2^\xc0\xaeh\x13\x98\x17\x85\xb2m\x93\xd5}h\x9e\xe3', b'\xd2\xeb  \x19\x87\xd9\xc1w\x19\x99}\x1c<\x06\xc9$\xe9\xf9N-r\xf08Ay\xaea!\xc5\x94,', b'\xd4\x10\xfaK\x86\xf1\xdb\x9a:\x7f\x18\xf5\x8b\xddFR\xce\x08\x0b\xfaH\xffu7\x0f\xf5b\x92C\xf1\x9c\x8d']
[b'\x91\xa8N\x1e\xaf

In [4]:
import time

def signing_experiment(preimages, commitements, k, message, filter):
    # Experiment for signing time
    ijs = []

    start = time.time()
    signature = hbss_sign(message, preimages, k, ijs)
    end = time.time()
    signing_time = end - start

    false_positive = True
    fresh_preimages = 0
    for ij in ijs:
        if ij not in filter:
            false_positive = False
            filter.append(ij)
            fresh_preimages += 1

    start = time.time()
    assert hbss_verify(message, signature, commitements, k)
    end = time.time()
    verification_time = end - start

    return signing_time, verification_time, false_positive, fresh_preimages

def signing_experiment_plus(preimages, commitements, l, k, beta, message, filter):
    # Experiment for signing time
    ijs = []

    start = time.time()
    signature = hbss_plus_sign(message, preimages, k, l, beta, ijs)
    end = time.time()
    signing_time = end - start

    false_positive = True
    fresh_preimages = 0
    for ij, d in ijs:
        if ij not in filter:
            false_positive = False
            filter[ij] = d
            fresh_preimages += 1
        else:
            if filter[ij] > d:
                false_positive = False
                filter[ij] = d
                fresh_preimages += 1

    start = time.time()
    assert hbss_plus_verify(message, signature, commitements, k, l, beta)
    end = time.time()
    verification_time = end - start

    return signing_time, verification_time, false_positive, fresh_preimages



In [5]:
import gc
import sys

# https://stackoverflow.com/questions/13530762/how-to-know-bytes-size-of-python-object-like-arrays-and-dictionaries-the-simp
def get_obj_size(obj):
    marked = {id(obj)}
    obj_q = [obj]
    sz = 0

    while obj_q:
        sz += sum(map(sys.getsizeof, obj_q))
        all_refr = ((id(o), o) for o in gc.get_referents(*obj_q))
        new_refr = {o_id: o for o_id, o in all_refr if o_id not in marked and not isinstance(o, type)}
        obj_q = new_refr.values()
        marked.update(new_refr.keys())

    return sz

In [6]:
from math import log, e 
from random import randbytes

m = 2 ** 23
key_length = 512
n = 2 ** 12
k = 512
filter = []

m_plus = 2 ** 19
key_length_plus = 512
k_plus = 1024

filter_plus = {}
beta = [0.3, 0.21, 0.14699999999999996, 0.10289999999999998, 0.07202999999999998, 0.05042099999999998, 0.035294699999999984, 0.02470628999999999, 0.01729440299999999, 0.012106082099999993, 0.008474257469999994, 0.005931980228999996, 0.0041523861602999965, 0.0029066703122099975, 0.002034669218546998, 0.0014242684529828986, 0.000996987917088029, 0.0006978915419616202, 0.0004885240793731341, 0.00034196685556119386]
l = 20

print(k)
print("Prob of false positive for HBSS:", (1 - e ** (-k * (n + 0.5) / (m - 1))) ** k)


signing_time_total = 0
verification_time_total = 0

signing_time_plus_total = 0
verification_time_plus_total = 0

experiments = 10

start = time.time()
preimages_plus, commitements_plus = hbss_plus_keygen(m_plus, l, key_length)
keygen_plus_time = time.time() - start

start = time.time()
preimages, commitements = hbss_keygen(m, key_length)
keygen_time = time.time() - start

print("Keygen time for HBSS:", keygen_time)
print("Keygen time for HBSS+:", keygen_plus_time)

fps = 0
fps_plus = 0

fresh_vals = 0
fresh_vals_plus = 0

for ix in range(experiments):
    message = str(ix) + randbytes(32).hex()

    signing_time, verification_time, fp, fresh = signing_experiment(preimages, commitements, k, message, filter)
    signing_time_plus, verification_time_plus, fp_plus, fresh_plus = signing_experiment_plus(preimages_plus, commitements_plus, l, k_plus, beta, message, filter_plus)

    signing_time_total += signing_time
    verification_time_total += verification_time

    signing_time_plus_total += signing_time_plus
    verification_time_plus_total += verification_time_plus

    fps += int(fp)
    fps_plus += int(fp_plus)

    fresh_vals += fresh
    fresh_vals_plus += fresh_plus

print("Total signing time for HBSS:", signing_time_total)
print("Total verification time for HBSS:", verification_time_total)

print("Total signing time for HBSS+:", signing_time_plus_total)
print("Total verification time for HBSS+:", verification_time_plus_total)

print("Average signing time for HBSS:", signing_time_total/experiments)
print("Average verification time for HBSS:", verification_time_total/experiments)

print("False positives for HBSS:", fps)
print("False positives for HBSS+:", fps_plus)

print("Average fresh values for HBSS:", fresh_vals/experiments)
print("Average fresh values for HBSS+:", fresh_vals_plus/experiments)

print("Fresh values for last experiment for HBSS:", fresh)
print("Fresh values for last experiment for HBSS+:", fresh_plus)

print("Key size for HBSS: (bytes)", get_obj_size(commitements))
print("Key size for HBSS+: (bytes)", get_obj_size(commitements_plus))


512
Prob of false positive for HBSS: 0.0
Keygen time for HBSS: 65.97584557533264
Keygen time for HBSS+: 21.818665027618408
Total signing time for HBSS: 0.011536359786987305
Total verification time for HBSS: 0.020817995071411133
Total signing time for HBSS+: 0.17783117294311523
Total verification time for HBSS+: 0.08054876327514648
Average signing time for HBSS: 0.0011536359786987304
Average verification time for HBSS: 0.0020817995071411135
False positives for HBSS: 0
False positives for HBSS+: 0
Average fresh values for HBSS: 511.8
Average fresh values for HBSS+: 1019.3
Fresh values for last experiment for HBSS: 511
Fresh values for last experiment for HBSS+: 1015
Key size for HBSS: (bytes) 1768182264
Key size for HBSS+: (bytes) 111088568


In [7]:
def prob_experiment(m, key_length, k, n, preimages = None):
    # Experiment for probability distribution
    if preimages is None:
        preimages, _ = hbss_keygen(m, key_length)
    filter = []

    for i in range(n):
        # genarate n signatures
        message = str(i) + randbytes(32).hex()
        ijs = []
        signature = hbss_sign(message, preimages, k, ijs)
        for ij in ijs:
            if ij not in filter:
                filter.append(ij)


    ijs = []
    message = str(n) + randbytes(32).hex()
    signature = hbss_sign(message, preimages, k, ijs)

    # get fresh preimages for n+1
    false_positive = True
    fresh_preimages = 0
    for ij in ijs:
        if ij not in filter:
            false_positive = False
            filter.append(ij)
            fresh_preimages += 1

    return false_positive, fresh_preimages
        
    
def prob_experiment_plus(m, key_length, l, k, beta, n, preimages = None):
    # Experiment for probability distribution
    if preimages is None:
        preimages, _ = hbss_plus_keygen(m, l, key_length)
    filter = {}

    for i in range(n):
        # genarate n signatures
        message = str(i) + randbytes(32).hex()
        ijs = []
        signature = hbss_plus_sign(message, preimages, k, l, beta, ijs)
        for ij, d in ijs:
            if ij not in filter:
                filter[ij] = d
            else:
                if filter[ij] > d:
                    filter[ij] = d

    ijs = []
    message = str(n) + randbytes(32).hex()
    signature = hbss_plus_sign(message, preimages, k, l, beta, ijs)

    # get fresh preimages for n+1
    false_positive = True
    fresh_preimages = 0
    for ij, d in ijs:
        if ij not in filter:
            false_positive = False
            filter[ij] = d
            fresh_preimages += 1
        else:
            if filter[ij] > d:
                filter[ij] = d
                fresh_preimages += 1

    return false_positive, fresh_preimages

In [8]:
experiments = 1000

n = 2 ** 5 - 2 ** 3
m = 2 ** 8
key_length = 512
k = int(m/n * log(2, e))

print(k)
print("Theoretical Probability of false positive for HBSS:", (1 - e ** (-k * (n + 0.5) / (m - 1))) ** k)

m_plus = 2 ** 16
key_length_plus = 512
k_plus = int(m/n * log(2, e))

l = 2
beta = [0.8, 0.2]

false_positives = 0
false_positives_plus = 0

fresh_vals = 0
fresh_vals_plus = 0

preimages_plus, _ = hbss_plus_keygen(m_plus, l, key_length_plus)
preimages, _ = hbss_keygen(m, key_length)

for ix in range(experiments):
    false_positive, fresh = prob_experiment(m, key_length, k, n, preimages)
    false_positive_plus, fresh_plus = prob_experiment_plus(m_plus, key_length_plus, l, k_plus, beta, n, preimages_plus)

    false_positives += int(false_positive)
    false_positives_plus += int(false_positive_plus)

    fresh_vals += fresh
    fresh_vals_plus += fresh_plus

print("EXPERIMENTS FOR N+1th SIGNATURE")

print("False positives for HBSS:", false_positives)
print("False positives for HBSS+:", false_positives_plus)

print("Average fresh values for HBSS:", fresh_vals/experiments)
print("Average fresh values for HBSS+:", fresh_vals_plus/experiments)

print("Probability of false positive for HBSS:", false_positives/experiments)
print("Probability of false positive for HBSS+:", false_positives_plus/experiments)


7
Theoretical Probability of false positive for HBSS: 0.00674300265862382


EXPERIMENTS FOR N+1th SIGNATURE
False positives for HBSS: 11
False positives for HBSS+: 0
Average fresh values for HBSS: 3.598
Average fresh values for HBSS+: 6.976
Probability of false positive for HBSS: 0.011
Probability of false positive for HBSS+: 0.0
